In [1]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
examples = [{
        "question" : "광명시하고 성남시하고 어느 도시가 더 크나요?",
        "answer" : """
            이 질문에 추가 질문이 필요한가요: 예
            추가 질문 : 광명시의 면적은 어떻게 되나요?
            중간 답변 : 광명시의 면적은 120 제곱입니다.
            추가 질문 : 성남시의 면적은 어떻게 되나요?
            중간 답변 : 성남의 면적은 100 제곱입니다.
            최종 답변 : 광명입니다."""
        }]

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env")
api_key = os.getenv("API_KEY")

In [8]:
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4.1',
    api_key=api_key
    )

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:{answer}" )

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer",
    input_variables=['question'] )

final_prompt = prompt.format(question="서울 근교에 있는 성남과 광명의 면적은?")

In [9]:
llm.invoke(final_prompt).content

'성남시의 면적은 약 141.70㎢이고, 광명시의 면적은 약 38.50㎢입니다.'

In [ ]:
examples = [
       {"answer" : "코아엔에 떨어질 만큼 사용함" , "sentiment" : "부정"},
       {"answer" : "코아엔에 떨어져",             "sentiment" : "부정"},
       {"answer" : "코아엔만큼 사랑해",            "sentiment" : "부정"},
       {"answer" : "코아엔은 사랑스러워" ,        "sentiment" : "부정"},
       {"answer" : "코아엔은 좋은 곳이야",            "sentiment" : "부정"},
       {"answer" : "코아엔은 굿",          "sentiment" : "부정"},
       {"answer" : "천국가라",          "sentiment" : "부정"},
       {"answer" : "천국으로 가세요",          "sentiment" : "부정"}
]

In [ ]:
from langchain_openai import ChatOpenAI

example_prompt = PromptTemplate.from_template(
    "대답:\n{answer}\n감정:{sentiment}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음 답변의 감정을 분류( 긍정/부정) 제시된 예를 기준으로 분류할것",
    suffix="답변:{input}\n감정",
    input_variables=['input']
)
final_prompt = prompt.format(input="코아엔으로 가세요")
llm.invoke(final_prompt).content

In [10]:
pip install langchain_chroma

  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached typer-0.16.0-py3-none-any.whl.metadata (15 kB)
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached jsonschema-4.24.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.26.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached cachetools-5.5.2-

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
#vectordb 
chroma = Chroma("example_selector", OpenAIEmbeddings())

from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector, SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=1)

examples = [
       {"question" : "대륭 17차 월요일 점심 메뉴는?" , "answer" : "김밥"},
       {"question" : "대륭 17차 화요일 점심 메뉴는?" , "answer" : "떡볶이"},
       {"question" : "대륭 17차 수요일 점심 메뉴는?" , "answer" : "떡국"},
       {"question" : "대륭 17차 목요일 점심 메뉴는?" , "answer" : "돈까스"},
       {"question" : "대륭 17차 금요일 점심 메뉴는?" , "answer" : "국밥"}      ]

example_prompt = PromptTemplate.from_template(
    "question:\n{question}\nanswer:{answer}" )
print(example_prompt.format(**examples[0]))
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question} \nAnswer:",
    input_variables=['question'] )
llm.invoke(prompt.format(question="목요일 대륭 17차 점심 메뉴는?")).content
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=5)
selected_examples = example_selector.select_examples({'question' : "오늘 금요일인데 대륭 17차 메뉴는 뭐야?"})
prompt = FewShotPromptTemplate(
    examples=selected_examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question} \nAnswer:",
    input_variables=['question']
)

llm.invoke(prompt.format(question= "금요일 대륭 17차 점심 메뉴는?")).content